In [1]:
from websockets.sync.client import connect
import json

ModuleNotFoundError: No module named 'websockets'

In this example we use the websockets library (https://websockets.readthedocs.io/en/stable/)

This is a super simple way to receive the websocket data, but feel free to use other methods

In [ ]:
def receive_data():
    max_msg = 10 # Feel free to change this or just let it loop forever. Maybe best to keep it low while testing...
    with connect("ws://192.87.172.71:1338") as websocket:
        count = 0
        while count < max_msg:
            msg = websocket.recv()
            try:
                msg = json.loads(msg)
                handle_message(msg) # Call function to do the actual data handling
            except json.JSONDecodeError:
                print("Failed to decode json, assuming next packet will be ok...")
                pass
            except Exception as e:
                # Assume something went wrong and stop receiving
                print("Something went horribly wrong!")
                print("Error:", e)
                break
            
            count += 1
        print("Received %d messages!" % count)

# 'Packet' format
The websocket will give you one entry per packet that was received on a receiver.

The packet can have the following fields:

- `address`: The address of the airplane that transmitted this message; you can use the `address` field to match different messages for the same aircraft,
- `altitude`: The altitude (in feet [https://en.wikipedia.org/wiki/Foot_(unit)]) of the airplane,
- `latitude`: The latitude of the airplane,
- `longitude`: The longitude of the airplane,
- `speed`: The speed (in knots [https://en.wikipedia.org/wiki/Knot_(unit)]) of the airplane,
- `heading`: The heading (in degrees) of the airplane,
- `callsign`: The callsign of the airplane
- `timestamp`: The time at which the receiver received the message. Note that this will not be exactly the same on both receivers due to processing delays and the clocks not being exactly in sync!
- `rssi`: The received signal strength indication of the message as determined by the receiver,
- `receiver`: The name of the receiver that received this message (either 'zi-5067' or 'zi-5110')

Note that due to there being two receivers this also means that some packets will be received twice, once on each receiver! You can detect this by noticing that some of the fields will be identical. Only the `timestamp`, `rssi` and `receiver` fields will be different.

There are 4 different 'types' of messages you will need to handle:
- The 'location' message: Has the `altitude`, `latitude` and `longitude` information,
- The 'altitude' message: This only has the `altitude` field,
- The 'speed' message: Has the `speed` and `heading` information,
- The 'callsign' message: Has the `callsign` information.

In [ ]:
def handle_message(msg):
    print(msg)
    if 'altitude' in msg: # check if this is one of the two types with altitude info
        if msg['altitude'] * 0.3048 >= 10000:
            print("Plane {} is at 10km or higher!".format(msg['address']))

In [ ]:
receive_data()